# NLP Information Extraction: Data Exploration

### Введение
В Контуре мы много работаем с документами: арбитражные иски, госзакупки, исполнительные производства. В данном задании мы предлагаем вам сделать модель, которая поможет отделу госзакупок извлекать 
нужный кусок текста из документа для того, чтобы сформировать анкету заявки. То, какой именно фрагмент текста нужно извлечь, зависит от пункта анкеты, соответствующего документу.
Всего в каждом документе, с которыми вы будет работать, есть 1 из 2-х пунктов анкеты, по которым необходимо извлекать кусочки из текста:
- обеспечение исполнения контракта
- обеспечение гарантийных обязательств

Соответственно, ваша модель, принимая на вход `текст документа` и `наименование одного из двух пунктов`, должна возвращать `соответствующий кусочек текста из текста документа`.

### Данные

##### train.json 
Данные для обучения в формате json имеют следующие поля:
- `id`: int - id документа
-  `text`: str - текст документа, в котором может содержаться фрагмент текста, соответствующий пункту анкеты из поля `label`
- `label`: str - название пункта анкеты. Может принимать одно из двух значений: `обеспечение исполнения контракта` или `обеспечение гарантийных обязательств`
- `extracted_part`: dict следующего формата:
```
    {
        'text': [фрагмент текста из поля `text`, соответствующий пункту анкеты], 
        'answer_start': [индекс символа начала фрагмента текста в тексте документа],
        'answer_end': [индекс символа конца фрагмента текста в тексте документа]
    }
```

##### test.json

Для демонстрации работы модели используйте данные из файла `test.json`. В нем есть все те же поля, что и в файле `train.json`, кроме поля `extracted_part` - именно его вам и нужно будет добавить,
для того, чтобы мы смогли оценить качество вашей модели.

### Тестовое задание

Для выполнения тестового задания требуется разработать модель, которая будет способна по паре `текст документа` и `пункт анкеты` извлекать из текста документа нужный фрагмент текста. 
Обучив модель, добавьте в файл `test.json` поле `extracted_part` в том же формате, что и в файле `train.json`. Новый файл назовите `predictions.json`

**Подсказка**: изучив данные, вы можете заметить, что у части наблюдений отсутствует фрагмент текста к извлечению (пустая строка внутри поля `extracted_part` с `answer_start` и
`answer_end` равными нулю). Это означает, что в тексте документа нет нужного фрагмента текста, соответствующего пункту анкеты. Учтите это в обучении вашей модели и при формировании
файла с ответами.

### Критерии оценки
1. Для оценки финального решения будет использоваться метрика `Accuracy`: доля наблюдений, в которых извлеченный моделью фрагмент текста полностью соответствует фактически
   требуемому фрагменту.
2. Чистота кода, оформление и понятность исследования.

### Требования к решению
В качестве решения мы ожидаем zip-архив со всеми *.py и *.ipynb файлами в папке solution и файлом `predictions.json` в корне. Формат имени zip-архива: LastName_FirstName.zip (пример Ivanov_Ivan.zip).
Файл `predictions.json` должен включать в себя колонки `id`, `text`, `label`, содержащие те же данные, что и исходный файл `test.json`, а также колонку `extracted_part` в том же
формате, что и в файле `train.json`
Разметка тестового набора данных и включение его в обучение/валидацию запрещены.

В папке solution должно быть отражено исследование и весь код, необходимый для воспроизведения исследования.

Успехов!

In [3]:
from tqdm import tqdm, trange
from pprint import pprint
import pandas as pd
import numpy as np
import json
import re
import random
import torch
import gc

In [8]:
with open(f'{DATA_DIR}/train.json', 'rb') as f:
    train = json.load(f)
with open(f'{DATA_DIR}/test.json', 'rb') as f:
    test = json.load(f)

In [9]:
LABELS = {
    'обеспечение исполнения контракта': {
        'short': 'CE',
        'full': 'CONTRACT-ENFORCEMENT',
        'id': 1,
        'emb': [],
        'emb_tuned': [],
        'regex_pattern': r'',
    },
    'обеспечение гарантийных обязательств': {
        'short': 'WO',
        'full': 'WARRANTY-OBLIGRATIONS',
        'id': 2,
        'emb': [],
        'emb_tuned': [],
        'regex_pattern': r'',
    },
}
LABELS_INV = {
    'CE': 'обеспечение исполнения контракта',
    'WO': 'обеспечение гарантийных обязательств',
}


In [10]:
EMB_DIM = 300  # natasha

## Data samples

In [ ]:
pprint(train[1701])

{'extracted_part': {'answer_end': [1410],
                    'answer_start': [1225],
                    'text': ['Размер обеспечения гарантийных обязательств '
                             'установлен в размере 20% от НМЦД: 1 644 839,76 '
                             'рублей. Гарантийные обязательства обеспечиваются '
                             'внесением денежных средств участником закупки']},
 'id': 611794687,
 'label': 'обеспечение гарантийных обязательств',
 'text': 'УТВЕРЖДАЮ Генеральный директор АО «САБ по уборке г. Курска» «07» '
         'сентября 2022 г. _____________________ А.Р. Зинатулин М.П. '
         'ДОКУМЕНТАЦИЯ ОБ АУКЦИОНЕ В ЭЛЕКТРОННОЙ ФОРМЕ, УЧАСТНИКАМИ КОТОРОГО '
         'МОГУТ БЫТЬ ТОЛЬКО СУБЪЕКТЫ МАЛОГО  5 апреля 2013 года N 44-ФЗ "О '
         'контрактной системе в сфере закупок товаров, работ, услуг для '
         'обеспечения государственных и муниципальных нужд". 54. Требования к '
         'участникам такой закупки и привлекаемым ими субподрядчикам, '


In [ ]:
pprint(test[0])

{'id': 762883279,
 'label': 'обеспечение исполнения контракта',
 'text': 'МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание услуг по техническому '
         'обслуживанию и ремонту принтеров и многофункциональных устройств, '
         'заправке и восстановлению картриджей (идентификационный код закупки '
         '223861800296886010100100590019511244) г. Ханты-Мансийск «___» '
         '____________ 2022 г.  (или) возмещения убытков причинённых '
         'Исполнителем убытков. 6. Обеспечение исполнения контракта 6.1. '
         'Исполнение контракта обеспечиваются предоставлением независимой '
         'гарантии, выданной банком и соответствующей требованиям Федерального '
         'закона от 05.04.2013 № 44-ФЗ «О контрактной системе в сфере закупок '
         'товаров, работ, услуг для обеспечения государственных и '
         'муниципальных нужд», или внесением денежных средств на указанный '
         'заказчиком счет, на котором в соответствии с законодательством '
         'Российской Фе

## Analysis of extracted parts

In [ ]:
for i, d in enumerate(train):
    d_text = d['text']
    e_text = d['extracted_part']['text'][0]
    answer_start = d['extracted_part']['answer_start'][0]
    answer_end = d['extracted_part']['answer_end'][0]
    padding = 5
    if e_text:
        print(f'[{i}] doc: {d_text[answer_start-padding:answer_end+padding]}')
        print(f'[{i}] txt: {" "*padding}{e_text}{" "*padding}')

[0] doc: акта Размер обеспечения исполнения контракта 6593.25 Российский рубль Поря
[0] txt:      Размер обеспечения исполнения контракта 6593.25 Российский рубль     
[1] doc: 005. Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта. В сл
[1] txt:      Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.     
[2] doc: акта Размер обеспечения исполнения контракта 10.00% Поря
[2] txt:      Размер обеспечения исполнения контракта 10.00%     
[3] doc: акта Размер обеспечения исполнения контракта 10.00% Поря
[3] txt:      Размер обеспечения исполнения контракта 10.00%     
[4] doc: акта Размер обеспечения исполнения контракта 10.00% Поря
[4] txt:      Размер обеспечения исполнения контракта 10.00%     
[5] doc: 7.4. Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ________
[5] txt:      Размер обеспечения исполнени

## Statistics and checks

To identify some statistics (number, top tokens, etc.), I will use popular NLP tools such as `natasha` and `spaCy`. Both python libraries have Russian language support and high scores on various NLP benchmarks. The default dictionaries, tokenizers, embeddings of words/sentences and models will be enough for counting and checking.

We can immediately say due to the specifics of the task (formal documents, contracts):
- The extracted text fragment is almost always a sentence or part of it
- The extracted text fragment almost always contains words from the name of the label, numbers, % and currency mention

In [11]:
from natasha import Segmenter, Doc, NewsEmbedding
from collections import Counter
import spacy

In [12]:
PUNCTUATION_PATTERN = r"[%.,!:?\-\(\)«»\'\"_№/]"

In [13]:
!python -q -m spacy download ru_core_news_lg

2023-05-03 12:17:51.080726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [14]:
emb = NewsEmbedding()
segmenter = Segmenter()

nlp = spacy.load("ru_core_news_lg", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
nlp.add_pipe("sentencizer")

multiple_annotations = False
ext_contains_quotes = False
tokens = []
tokens_spacy = []
tokens_e_spacy = []
sents = []
sents_spacy = []
words = Counter()
words_spacy = Counter()
ANSWER_WORDS = Counter()
answer_words_spacy = Counter()

for d in train + test:
  if 'extracted_part' in d:
    ext_text = d['extracted_part']['text'][0]
    if len(d['extracted_part']['text']) > 1:
      multiple_annotations = True
      print('Multiple annotations: ', d)
    if re.search(r'[\'|"|«].*?[\'|"|»]', ext_text):
      ext_contains_quotes = True
      print('Quotes: ', d)
    # natasha
    doc = Doc(ext_text)
    doc.segment(segmenter)
    ANSWER_WORDS.update([_.text for _ in doc.tokens])
    # spacy
    doc = nlp(ext_text)
    answer_words_spacy.update([_.text for _ in doc])
    # spacy for ext
    if d['extracted_part']['text'][0]:
      doc = nlp(d['extracted_part']['text'][0])
      tokens_e_spacy.append(len(doc))
  # natasha
  doc = Doc(d['text'])
  doc.segment(segmenter)
  tokens.append(len(doc.tokens))
  words.update([_.text for _ in doc.tokens])
  sents.append(len(doc.sents))
  # spacy
  doc = nlp(d['text'])
  tokens_spacy.append(len(doc))
  words_spacy.update([_.text for _ in doc])
  sents_spacy.append(len(list(doc.sents)))

print(f'Multiple annotations: {multiple_annotations}')
print(f'Quotes in answer: {ext_contains_quotes}')
print(f'Tokens natasha (number): max. {max(tokens)}, mean {sum(tokens)/len(tokens):.2f}, min. {min(tokens)}')
print(f'Tokens spacy (number): max. {max(tokens_spacy)}, mean {sum(tokens_spacy)/len(tokens_spacy):.2f}, min. {min(tokens_spacy)}')
print(f'Tokens extracted part spacy (number): max. {max(tokens_e_spacy)}, mean {sum(tokens_e_spacy)/len(tokens_e_spacy):.2f}, min. {min(tokens_e_spacy)}')
print(f'Sentences natasha (number): max. {max(sents)}, mean {sum(sents)/len(sents):.2f}, min. {min(sents)}')
print(f'Sentences spacy (number): max. {max(sents_spacy)}, mean {sum(sents_spacy)/len(sents_spacy):.2f}, min. {min(sents_spacy)}')

print(f'Топ 15 токенов natasha: {words.most_common(15)}')
print(f'Топ 15 токенов spacy: {words_spacy.most_common(15)}')
print(f'Топ токенов без пунктуации natasha: {[v for v in words.most_common(30) if not re.search(PUNCTUATION_PATTERN, v[0])]}')
print(f'Топ токенов без пунктуации spacy: {[v for v in words_spacy.most_common(30) if not re.search(PUNCTUATION_PATTERN, v[0])]}')
print(f'Топ 15 токенов в извлеченной части natasha: {ANSWER_WORDS.most_common(15)}')
print(f'Топ 15 токенов в извлеченной части spacy: {answer_words_spacy.most_common(15)}')
print(f'Топ токенов без пунктуации в извлеченной части natasha: {[v for v in ANSWER_WORDS.most_common(30) if not re.search(PUNCTUATION_PATTERN, v[0])]}')
print(f'Топ токенов без пунктуации в извлеченной части spacy: {[v for v in answer_words_spacy.most_common(30) if not re.search(PUNCTUATION_PATTERN, v[0])]}')

С кавычками:  {'id': 389142502, 'text': 'Общая часть 1. вки остановочных павильонов, утвержденном «Заказчиком». 1.15. Оценка качества работ проводится путем визуального осмотра поставленных и установленных остановочных павильонов представителями Заказчика и Поставщика. В случае выявления работы, не соответствующей требованиям, указанным в настоящем техническом задании, Заказчик направляет в адрес Поставщика предписание об устранении выявленных нарушений с указанием сроков их исполнения. Не выполнение предписания в установленные сроки влечет за собой применение к Поставщику штрафных санкций. 2. Объем и сроки предоставления гарантий качества поставляемого товара 2.1. Поставщик гарантирует качество поставляемого товара, функционирование объектов установки остановочных павильонов и входящих в них материалов и оборудования в соответствии с существующими стандартами, в течение 3 лет (с даты окончания срока поставки и установки остановочных павильонов), за исключением случаев повреждения уста

It is not necessary to process several annotations (i.e. there are no overlapping annotations as well), which means that the task can be represented as finding named entities in the text. There is an idea to clean the text from the text in quotes, large numbers, underscores, but all this creates a problem with shifting the indexes of the extracted text fragment.

The documents contain a large number of tokens (>512), so we will have to take this into account when training the model (the option to split the document creates a problem with the indexes of the extracted text fragment). Building an index of embeddings of document sentences will be very fast, since there are an average of 9 sentences in each document, which also means faster training of a transformer working with embeddings of sentences.

The tops of the `natasha` and `spaCy` tokens differ quite slightly (I compared them for the sake of interest, because I'm going to use both tools). It can be seen that all the words from the label are in the top of the words of the document texts, which means that there may be several applications containing them in the documents, and we need to teach the model to identify exactly the fragment we need, for example, by adding more tokens for the label. To analyze this, I have compiled the top tokens in the extracted part and plan to use it to modify the embedding of the label. Also, at first glance, it seems that it is still worth including punctuation symbols (%, brackets) in the creation of the embedding of the label, because this is exactly what will help to identify the most appropriate text fragment in the document.

## Label embeddings and search pattern

In [15]:
from natasha import Segmenter, Doc, NewsEmbedding, MorphVocab, NewsMorphTagger

In [16]:
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)


def emb_sentence(sentence):
  d = Doc(sentence)
  d.segment(segmenter)
  e = np.array([emb[token.text] for token in d.tokens if token.text in emb]).mean(axis=0)
  return e if e.shape == (EMB_DIM,) else None


def get_label_span_from_text(text, extracted_part):
  start = text.find(extracted_part)
  return start, start + len(extracted_part)

In [17]:
for label in LABELS:
  LABELS[label]['emb'] = emb_sentence(label)

  most_common_tokens = ' '.join([v[0] for v in ANSWER_WORDS.most_common(20) if not re.search(PUNCTUATION_PATTERN, v[0])])
  LABELS[label]['emb_tuned'] = emb_sentence(most_common_tokens)

  d = Doc(label)
  d.segment(segmenter)
  d.tag_morph(morph_tagger)
  for token in d.tokens:
    token.lemmatize(morph_vocab)
  lemmas = [_.lemma for _ in d.tokens]
  lemmas_pattern = '|'.join(lemmas)
  LABELS[label]['regex_pattern'] = lemmas_pattern

In [18]:
LABELS

{'обеспечение исполнения контракта': {'short': 'CE',
  'full': 'CONTRACT-ENFORCEMENT',
  'id': 1,
  'emb': array([ 5.12298346e-01, -3.32074016e-01,  4.50839614e-03, -5.33237875e-01,
          1.34650484e-01, -7.13107362e-02, -3.13343167e-01,  1.38494372e-02,
          4.15200591e-01, -5.87544858e-01,  4.01647901e-03, -1.21317744e-01,
          1.91846285e-02,  1.80541709e-01, -1.34432673e-01, -3.64194185e-01,
          7.50907278e-03, -1.90318331e-01,  1.53998137e-01,  7.81209171e-02,
          1.46370530e-02, -4.34108883e-01, -5.05719423e-01,  1.85416102e-01,
         -2.52503399e-02,  4.91171069e-02,  1.70502782e-01,  2.42485091e-01,
          2.21013203e-01, -8.34056213e-02,  3.72210652e-01, -2.41341829e-01,
          1.75793126e-01, -2.52902985e-01,  2.34050155e-01, -1.58185020e-01,
          1.44518092e-01, -4.82092500e-02, -1.84709847e-01, -1.65039256e-01,
          8.04059058e-02, -2.87849069e-01,  2.42740419e-02, -6.18132688e-02,
          5.58322854e-02, -4.70985956e-02,  1.32

## Statistics on extracted parts

In [ ]:
from natasha import Segmenter, Doc, NewsEmbedding, MorphVocab, NewsMorphTagger


emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(emb)


def create_lemmatized_doc(text):
  d = Doc(text)
  d.segment(segmenter)
  d.tag_morph(morph_tagger)
  for token in d.tokens:
    token.lemmatize(morph_vocab)
  return d, ' '.join([token.lemma for token in d.tokens])


def search_for_direct_match(text, label):
  _, text_lemmatized = create_lemmatized_doc(text)
  matches = re.findall(LABELS[label]['regex_pattern'], text_lemmatized)

  return True if matches else False

In [ ]:
from collections import Counter
from tqdm import tqdm


counter = Counter()
for d in tqdm(train):
    d_text = d['text']
    label = d['label']
    if search_for_direct_match(d_text, label):
        counter.update([label])
counter

100%|██████████| 1799/1799 [01:47<00:00, 16.78it/s]


Counter({'обеспечение исполнения контракта': 985,
         'обеспечение гарантийных обязательств': 589})

In [ ]:
for label in LABELS:
  overall_count = len([_ for _ in train if _['label'] == label])
  percent = counter[label]/overall_count * 100
  print(f'[{label}]: {counter[label]}/{overall_count} ({percent:.2f}%) have direct match of word from label in extracted part')

[обеспечение исполнения контракта]: 985/988 (99.70%) have direct match of word from label in extracted part
[обеспечение гарантийных обязательств]: 589/811 (72.63%) have direct match of word from label in extracted part
